## Installation

In [ ]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl==0.15.2 triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf datasets huggingface_hub hf_transfer
    !pip install --no-deps unsloth

## Unsloth Set Up

In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 8192 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

model, tokenizer = FastLanguageModel.from_pretrained(
    # And also all Instruct versions and Math. Coding verisons!
    model_name = "unsloth/Qwen2.5-7B",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
Unsloth: Failed to patch Gemma3ForConditionalGeneration.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.3.19: Fast Qwen2 patching. Transformers: 4.51.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors.index.json:   0%|          | 0.00/106k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

...

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/605 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/617 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2025.3.19 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


## Data Prep

In [ ]:
PROMPT = """You are an expert evaluator for the output of a large language model (LLM). Your job is to assess whether the generated response is appropriate and grounded, given the query and the context chunk from the source document.

Instructions:
Based on the provided context, determine whether the response:

    Accurately reflects information from the context chunk?

    Correctly and sufficiently answers the query?

    Avoids introducing information that is not in the context?

Output:
Respond with one of the following labels:

    YES – if the response is accurate, grounded in the context, and answers the query appropriately.

    NO – if the response is inaccurate, unrelated, or includes unsupported claims.

Input:

Query: {instruction}

LLM Response: {output}

Output: {label}
"""

In [ ]:
import pandas as pd
from datasets import Dataset

# Đọc dữ liệu từ file CSV
df = pd.read_csv('/content/evaluated_Data250421_Full.csv')  # Thay 'duong_dan_toi_file.csv' bằng đường dẫn thực tế

# Đổi tên cột để phù hợp với định dạng mong muốn
df = df.rename(columns={
    "query": "instruction",
    "chunk": "input",
    "response": "output",
    "check_response": "label"
})

# Tạo tập dữ liệu từ DataFrame
dataset = Dataset.from_pandas(df)

In [ ]:
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs = examples["input"]
    outputs = examples["output"]
    labels = examples["label"]
    texts = []
    for instruction, input_text, output, label in zip(instructions, inputs, outputs, labels):
        text = PROMPT.format(
            instruction=instruction,
            # input=input_text,
            output=output,
            label=label
        ) + tokenizer.eos_token  # Thêm EOS_TOKEN để kết thúc prompt
        texts.append(text)
    return {"text": texts, }

# Áp dụng hàm định dạng prompt
dataset = dataset.map(formatting_prompts_func, batched=True, )


Map:   0%|          | 0/383 [00:00<?, ? examples/s]

In [ ]:
dataset = dataset.remove_columns(
    [col for col in dataset.column_names if col != "text"]
)

In [ ]:
print(dataset)

Dataset({
    features: ['instruction', 'input', 'output', 'label', 'text'],
    num_rows: 383
})


In [ ]:
print(dataset[1]['text'])

You are an expert evaluator for the output of a large language model (LLM). Your job is to assess whether the generated response is appropriate and grounded, given the query and the context chunk from the source document.

Instructions:
Based on the provided context, determine whether the response:

    Accurately reflects information from the context chunk?

    Correctly and sufficiently answers the query?

    Avoids introducing information that is not in the context?

Output:
Respond with one of the following labels:

    YES – if the response is accurate, grounded in the context, and answers the query appropriately.

    NO – if the response is inaccurate, unrelated, or includes unsupported claims.

Input:

Query: tôi quên không điền form đăng ký lịch làm việc, tôi phải làm như nào ?

LLM Response: Nếu bạn quên không điền form đăng ký lịch làm việc, bạn cần thực hiện các bước sau:

1. **Đăng ký bổ sung:** Truy cập hệ thống WSM và đăng ký lịch làm việc theo đúng quy định. Lưu ý rằn

## Train Model

In [ ]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer, padding=True)

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        # num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
    ),
)

Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/383 [00:00<?, ? examples/s]

In [ ]:
# @title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.741 GB.
7.266 GB of memory reserved.


In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 383 | Num Epochs = 2 | Total steps = 60
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 40,370,176/7,000,000,000 (0.58% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,1.950800
2,1.988300
3,1.968200
4,1.890000
5,1.860900
6,2.026700
7,1.516700
8,1.468300
9,1.321700
10,1.160900


## Inferences

In [ ]:
# alpaca_prompt = Copied from above
# 12 minutes
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    PROMPT.format(
        instruction="Sun* có tổ chức khám sức khỏe định kỳ không?",
        # input=input_text,
        output="""- Bảo hiểm Sun*care áp dụng cho nhân viên ký hợp đồng lao động chính thức (mã B) từ trên 3 tháng đến không xác định thời hạn.

- Đối với người lao động ký hợp đồng lao động lương theo giờ, hợp đồng lao động xác định thời hạn từ 03 tháng trở xuống, hoặc người nước ngoài ký hợp đồng lao động với công ty: chỉ tham gia gói bảo hiểm tai nạn lao động theo quy định.""",
        label=""# output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
tokenizer.batch_decode(outputs)

['You are an expert evaluator for the output of a large language model (LLM). Your job is to assess whether the generated response is appropriate and grounded, given the query and the context chunk from the source document.\n\nInstructions:\nBased on the provided context, determine whether the response:\n\n    Accurately reflects information from the context chunk?\n\n    Correctly and sufficiently answers the query?\n\n    Avoids introducing information that is not in the context?\n\nOutput:\nRespond with one of the following labels:\n\n    YES – if the response is accurate, grounded in the context, and answers the query appropriately.\n\n    NO – if the response is inaccurate, unrelated, or includes unsupported claims.\n\nInput:\n\nQuery: Sun* có tổ chức khám sức khỏe định kỳ không?\n\nLLM Response: - Bảo hiểm Sun*care áp dụng cho nhân viên ký hợp đồng lao động chính thức (mã B) từ trên 3 tháng đến không xác định thời hạn.\n\n- Đối với người lao động ký hợp đồng lao động lương theo g

# Train with chunk

In [ ]:
PROMPT = """You are an expert evaluator for the output of a large language model (LLM). Your job is to assess whether the generated response is appropriate and grounded, given the query and the context chunk from the source document.

Instructions:
Based on the provided context, determine whether the response:

    Accurately reflects information from the context chunk?

    Correctly and sufficiently answers the query?

    Avoids introducing information that is not in the context?

Output:
Respond with one of the following labels:

    YES – if the response is accurate, grounded in the context, and answers the query appropriately.

    NO – if the response is inaccurate, unrelated, or includes unsupported claims.

Input:

Query: {instruction}

Chunk: {input}

LLM Response: {output}

Output: {label}
"""

In [ ]:
import pandas as pd
from datasets import Dataset

# Đọc dữ liệu từ file CSV
df = pd.read_csv('/content/evaluated_Data250421_Full.csv')  # Thay 'duong_dan_toi_file.csv' bằng đường dẫn thực tế

# Đổi tên cột để phù hợp với định dạng mong muốn
df = df.rename(columns={
    "query": "instruction",
    "chunk": "input",
    "response": "output",
    "check_response": "label"
})

# Tạo tập dữ liệu từ DataFrame
dataset = Dataset.from_pandas(df)

In [ ]:
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs = examples["input"]
    outputs = examples["output"]
    labels = examples["label"]
    texts = []
    for instruction, input_text, output, label in zip(instructions, inputs, outputs, labels):
        text = PROMPT.format(
            instruction=instruction,
            input=input_text,
            output=output,
            label=label
        ) + tokenizer.eos_token  # Thêm EOS_TOKEN để kết thúc prompt
        texts.append(text)
    return {"text": texts, }

# Áp dụng hàm định dạng prompt
dataset = dataset.map(formatting_prompts_func, batched=True, )


Map:   0%|          | 0/383 [00:00<?, ? examples/s]

In [ ]:
dataset = dataset.remove_columns(
    [col for col in dataset.column_names if col != "text"]
)

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        # num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
    ),
)

Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/383 [00:00<?, ? examples/s]

In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 383 | Num Epochs = 2 | Total steps = 60
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 40,370,176/7,000,000,000 (0.58% trained)


Step,Training Loss
1,1.212300
2,0.798300
3,1.095900
4,0.952600
5,0.677000
6,0.940900
7,1.094600
8,0.949900
9,0.971900
10,1.008400


In [ ]:
# alpaca_prompt = Copied from above
# 2 h 30 m in finetuning
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    PROMPT.format(
        instruction="Sun* có tổ chức khám sức khỏe định kỳ không?",
        input="""
        "{\"question\": \"Sun* có tổ chức khám sức khỏe định kỳ không?\", \"answer\": \"- Hằng năm, ngân sách cho việc khám sức khỏe định kỳ được xây dựng và điều chỉnh phù hợp với tình hình kinh doanh của Công ty và đảm bảo tuân theo quy định của pháp luật.\\n- Đối tượng áp dụng:\\n      + Toàn bộ NLĐ ký Hợp đồng với Công ty\\n      + Sinh viên thực tập nghề ký HĐ đào tạo với Công ty\\n- Tần suất: 1 lần/năm; 2 lần/năm (Đối với TH NLĐ cao tuổi/ khuyết tật)\\n- Tham khảo chi tiết tại:  1-200_HỆ THỐNG TỔNG ĐÃI NGỘ\\n(Điều 3.1. Chính sách khám sức khỏe định kỳ - phần B - Chương III - trang 6)\"}"
        """,
        output="""- Bảo hiểm Sun*care áp dụng cho nhân viên ký hợp đồng lao động chính thức (mã B) từ trên 3 tháng đến không xác định thời hạn.

- Đối với người lao động ký hợp đồng lao động lương theo giờ, hợp đồng lao động xác định thời hạn từ 03 tháng trở xuống, hoặc người nước ngoài ký hợp đồng lao động với công ty: chỉ tham gia gói bảo hiểm tai nạn lao động theo quy định.""",
        label=""# output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
tokenizer.batch_decode(outputs)